#**Projeto: Análise de Vendas no Varejo**

**Analisar dados de vendas do varejo para entender o comportamento dos clientes, o desempenho das categorias, o impacto dos descontos e padrões sazonais, preparando uma base confiável para visualização no Power BI.**

### **Exploração Inicial dos Dados (Raw)**

Dados são inspecionados para compreender a estrutura da tabela, as colunas disponíveis e identificar possíveis problemas de qualidade antes do tratamento.

In [0]:
%sql
select * from workspace.teste.store_sales
limit 10;

CustomerID,Age,Gender,Category,ItemPurchased,Amount,Season,PaymentMethod,ItemRating,DiscountApplied(%),PreviousPurchases
1,58,Female,Accessories,Handbag,115.5,Autumn,Card,3.5,18,4
2,40,Male,Mens Clothing,Shirt,103.43,Spring,Card,4.1,13,4
3,66,Female,Sports,Football,35.45,Spring,Card,3.3,11,3
4,39,Female,Accessories,Handbag,153.31,Spring,Card,4.4,13,4
5,23,Female,Home,Curtains,151.43,Winter,Card,4.1,20,10
6,48,Male,Groceries,Snacks,36.76,Autumn,Card,2.5,14,2
7,54,Male,Electronics,Mobile Phone,1396.16,Autumn,Card,4.7,18,4
8,59,Female,Beauty,Perfume,35.55,Summer,Card,4.6,14,7
9,51,Male,Electronics,Smart Watch,2238.79,Spring,Card,4.3,20,5
10,47,Female,Footwear,Formal Shoes,225.49,Spring,Card,4.7,18,4


## Avaliação da Qualidade dos Dados

Verificação da consistência dos dados, com foco na identificação de valores nulos e inválidos em colunas críticas.

In [0]:
%sql
DESCRIBE workspace.teste.store_sales;

col_name,data_type,comment
CustomerID,bigint,null
Age,bigint,null
Gender,string,null
Category,string,null
ItemPurchased,string,null
Amount,double,null
Season,string,null
PaymentMethod,string,null
ItemRating,double,null
DiscountApplied(%),bigint,null


##Limpeza e Tratamento dos Dados (Silver)

Dados são filtrados e tratados para garantir consistência e confiabilidade nas análises.

In [0]:
%sql
CREATE OR REPLACE TABLE retail_silver AS
SELECT
  CustomerID                              AS customer_id,
  CAST(Age AS INT)                       AS age,
  Gender                                 AS gender,
  Category                               AS category,
  ItemPurchased                          AS item_purchased,
  CAST(Amount AS DOUBLE)                 AS amount,
  Season                                 AS season,
  PaymentMethod                          AS payment_method,
  CAST(ItemRating AS DOUBLE)             AS item_rating,
  CAST(`DiscountApplied(%)` AS DOUBLE)   AS discount_applied_pct,
  CAST(PreviousPurchases AS INT)         AS previous_purchases
FROM teste.store_sales;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT
  COUNT(*) AS total_rows,
  COUNT(customer_id) AS customer_id_not_null,
  COUNT(age) AS age_not_null,
  COUNT(amount) AS amount_not_null
FROM retail_silver;

total_rows,customer_id_not_null,age_not_null,amount_not_null
5000,5000,5000,5000


In [0]:
%sql
SELECT *
FROM retail_silver
WHERE age <= 0 OR amount <= 0;

customer_id,age,gender,category,item_purchased,amount,season,payment_method,item_rating,discount_applied_pct,previous_purchases


In [0]:
df = spark.table("retail_silver")

In [0]:
df_clean = df.dropna(subset=["customer_id", "age", "amount"])

In [0]:
df_clean = df_clean.filter(
    (df_clean.age > 0) & (df_clean.amount > 0)
)

## Criação de Variáveis Analíticas e análises de negócio

Novas colunas são criadas para facilitar análises, visualizações e respostas de perguntas estratégicas do negócio.

In [0]:
from pyspark.sql.functions import when
df_clean = df_clean.withColumn(
    "age_group",
    when(df_clean.age < 25, "Under 25")
    .when(df_clean.age <= 40, "25-40")
    .when(df_clean.age <= 60, "41-60")
    .otherwise("60+")
)

In [0]:
%sql
SELECT
  COUNT(*) AS total_sales,
  COUNT(DISTINCT customer_id) AS total_customers,
  ROUND(SUM(amount), 2) AS total_revenue,
  ROUND(AVG(amount), 2) AS avg_ticket
FROM retail_silver;

total_sales,total_customers,total_revenue,avg_ticket
5000,5000,1425452.61,285.09


In [0]:
%sql
SELECT
  category,
  COUNT(*) AS total_sales,
  ROUND(SUM(amount), 2) AS revenue
FROM retail_silver
GROUP BY category
ORDER BY revenue DESC;

category,total_sales,revenue
Electronics,507,895402.35
Footwear,983,141142.24
Sports,835,115598.11
Womens Clothing,547,61718.41
Mens Clothing,511,59680.43
Accessories,470,47633.83
Home,286,46374.22
Beauty,529,43887.06
Groceries,332,14015.96


In [0]:
%sql
SELECT
  discount_applied_pct,
  COUNT(*) AS total_sales,
  ROUND(AVG(amount), 2) AS avg_ticket
FROM retail_clean
GROUP BY discount_applied_pct
ORDER BY discount_applied_pct;

discount_applied_pct,total_sales,avg_ticket
0.0,9,373.36
2.0,55,195.6
4.0,138,300.04
5.0,12,154.62
6.0,204,264.31
7.0,48,196.29
8.0,304,309.0
9.0,143,316.08
10.0,318,261.58
11.0,236,291.75


In [0]:
%sql
SELECT
  CASE
    WHEN discount_applied_pct > 0 THEN 'With Discount'
    ELSE 'No Discount'
  END AS discount_flag,
  COUNT(*) AS total_sales,
  ROUND(AVG(amount), 2) AS avg_ticket
FROM retail_silver
GROUP BY
  CASE
    WHEN discount_applied_pct > 0 THEN 'With Discount'
    ELSE 'No Discount'
  END;
  

discount_flag,total_sales,avg_ticket
With Discount,4991,284.93
No Discount,9,373.36


In [0]:
%sql
SELECT
  season,
  ROUND(SUM(amount), 2) AS revenue,
  ROUND(AVG(amount), 2) AS avg_ticket
FROM retail_silver
GROUP BY season;

season,revenue,avg_ticket
Autumn,343677.89,279.19
Spring,376029.4,289.25
Winter,354884.96,285.28
Summer,350860.36,286.42


In [0]:
%sql
SELECT
  payment_method,
  COUNT(*) AS total_sales,
  ROUND(AVG(amount), 2) AS avg_ticket
FROM retail_silver
GROUP BY payment_method
ORDER BY total_sales DESC;

payment_method,total_sales,avg_ticket
Card,4009,340.81
Cash on Delivery,991,59.7


## Preparação da Camada Final (Gold)

Após o tratamento e enriquecimento dos dados, a base final é consolidada e salva para consumo no Power BI.

In [0]:
df_clean.write.mode("overwrite").saveAsTable("retail_gold")